<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Ejercicio 1</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>estudiante: Lazcano Lagos Jhony Brandon</i></h5>
  <h5 align="center"><i>estudiante: Ramírez Montes Jonathan Natael</i></h5>
  <h5 align="center"><i>Análisis Multivariado</i></h5>
  </font>

1. El dataset de consumo plantas.csv contienen datos que corresponden a 17 plantas electricas y el consumo de agua en galones que estas generan mensualmente. donde 
* Y: Consumo de agua en galones, 
* X1: Temperatura media mensual en grados Fahrenheit, 
* X2: Produccion mensual en libras, 
* X3: Número de días de operacion mensual, 
* X4: Numero de empleados, 
* X5: No especificada. 

Se pretende explicar el consumo de agua (Y) en funcion de las demás variables.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

a) Ajuste un modelo de regresion lineal usando mínimos cuadrados y muestre la tabla con el resumen del ajuste. ¿Que observa en dicho ajuste?, ¿Como es el numero de condición y a que se debe?

In [2]:
rel_path = './datasets/'
plantas = pd.read_csv(rel_path + 'consumo_plantas.csv')

In [3]:
plantas.head()

,Planta,Y,X1,X2,X3,X4,X5
0,1,3067,58.8,7107,21,129,176.7
1,2,2828,65.2,6373,22,141,157.6
2,3,2891,70.9,6796,22,153,193.0
3,4,2994,77.4,9208,20,166,212.6
4,5,3082,79.3,14792,25,193,251.1


In [4]:
# Separando las predictoras del target
x = plantas[['X1', 'X2', 'X3', 'X4', 'X5']]
y = plantas[['Y']]

# anadiendo el termino cte
x = sm.add_constant(x)

# ajuste del modelo
result = sm.OLS(y, x).fit()

# resumen del modelo
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                     9.995
Date:                Thu, 08 Dec 2022   Prob (F-statistic):           0.000834
Time:                        17:41:14   Log-Likelihood:                -112.78
No. Observations:                  17   AIC:                             237.6
Df Residuals:                      11   BIC:                             242.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7707.9134   1423.389      5.415      0.0

C:\Users\jonat\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


De lo anterior, podemos deducir que la variavilidad explicada ($R^2$) por el modelo es del 82%, es decir un 82% del incremento en el consumo de agua está dado por el incremento de las $X_i$ y dado que la $R^2$ ajustada se aleja bastante de $R^2$ podríamos inferir que tenemos variables de más, Además esto se apoya también de los P_values y los intervalos de confianza, pues podemos observar que X_1, X_4 y X_5 tiene un p_value mayor al 5% y sus intervalos de confianza tocan al cero por lo que dichas variables serían candidatas a salir del modelo

El **número de condición** es grande y esto se debe a una posible multicolinealidad entre las variables, es decir que existe mas de una variable que está explicando lo mismo o que una columna podría ser una combinación lineal de otra

b) Estandarice los datos de las variables explicativas con escalador estandar, posteriormente ajuste un modelo de componentes principales con 5 componentes. Calcule la varianza explicada acumulada por cada componente, muestre la matriz de transformacion PCA. ¿Que observa? ¿A que cree que deba dicho fenomeno?. No use la variable a regresar (Y) para estandarizar ni en las componentes.

In [5]:
scaler = StandardScaler()

In [6]:
feat_cols = ['X1', 'X2', 'X3', 'X4', 'X5']

In [7]:
scaler.fit(x[feat_cols])

StandardScaler()

In [8]:
x_centered = scaler.transform(x[feat_cols])

In [9]:
kw = len(feat_cols)

In [10]:
pca_model_plant = PCA(n_components=kw).fit(x_centered)

In [11]:
#nombre de las columnas del pca
pca_x_cols = ['pc'+str(i+1) for i in range(0,kw)]

In [12]:
#matriz de transformación o cargas es el .components_
pca_x_matrix = pd.DataFrame(pca_model_plant.components_.transpose(),
                            columns = pca_x_cols[:kw])
pca_x_matrix['cols'] = feat_cols
print('Matriz de cargas'.center(50, '*'))
display(pca_x_matrix)

*****************Matriz de cargas*****************


,pc1,pc2,pc3,pc4,pc5,cols
0,0.188248,0.653083,0.679902,-0.060210,-0.268615,X1
1,-0.549779,0.248273,0.082817,0.749440,0.259970,X2
2,0.071478,0.681751,-0.718896,-0.059771,-0.098596,X3
3,-0.564651,0.182732,0.107221,-0.647975,0.465199,X4
4,-0.581692,-0.116906,-0.050660,-0.106161,-0.796323,X5


In [13]:
df_pca_plant = pd.DataFrame(pca_model_plant.transform(x_centered),
                            columns = pca_x_cols[:kw])

In [14]:
df_pca_plant

,pc1,pc2,pc3,pc4,pc5
0,3.200976,-1.202898,-0.395657,0.562092,-0.089517
1,3.364802,-0.309641,-0.508458,0.005754,0.218413
2,2.651303,0.025465,-0.177932,-0.366979,-0.165731
3,1.685424,-0.370732,1.276366,-0.220461,-0.003960
4,-0.092919,2.672824,-0.925997,-0.117923,0.012409
5,0.190905,1.789655,0.168702,0.069756,0.309971
6,0.458841,-0.461503,1.062354,0.050022,-0.124368
7,-0.137330,0.770500,-0.787184,-0.047961,0.066021
8,-0.710203,-1.232448,0.216068,-0.248984,0.049167
9,-1.507402,-1.985029,-0.578816,0.153720,-0.180928


In [15]:
pca_model_plant.explained_variance_ratio_

array([0.56723789, 0.30062477, 0.11080613, 0.01582018, 0.00551102])

Se puede observar que las dos primeras PCA's se llevan el 86% de la explicabilidad de la varianza, tomar 3 PCA's acumula un 97% de la varianzada de los datos, también se puede observar que X2 y X3 en la matriz de cargas presentan 2 de los valores más altos en la matriz de cargas, esto podría estar ligado a que estas 2 variables aportan un poco más a la explicabilidad de los datos.

c) Ajuste de nuevo un modelo de regresion lineal usando mínimos cuadrados pero ahora sobre las componentes principales, luego muestre la tabla con el resumen del nuevo ajuste. ¿Que diferencia observa en el nuevo ajuste respecto al primero en el numero de condición y coeficiente de determinación?. Investigue y argumente como es que las componentes principales ayudan a mitigar el problema de la multicolinealidad severa.

In [16]:
x_pca = sm.add_constant(df_pca_plant)

# ajuste del modelo
mod_pca = sm.OLS(y, x_pca).fit()

# resumen del modelo
print(mod_pca.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                     9.995
Date:                Thu, 08 Dec 2022   Prob (F-statistic):           0.000834
Time:                        17:41:15   Log-Likelihood:                -112.78
No. Observations:                  17   AIC:                             237.6
Df Residuals:                      11   BIC:                             242.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3303.7059     55.499     59.528      0.0

C:\Users\jonat\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Con el modelo sin PCA's teníamos 3 variables que cuyo p_value > 0.05 y con el modelo de PCA's ahora observamos que sólo hay 2.

En cuanto a la $R^2$ (coeficiente de determinación) y a la $R^2$ ajustada las métricas son las mismas (y tiene sentido, pues lo que se está haciendo con los datos es un cambio de base).

En este modelo el número de condición desaperece lo que indica la desaparición de la multicolinealidad. La **razón** por la cual la multicolinealidad desaperece es porque las componenentes principales son ortogonales entre sí, eso dado por las propiedades del álgebra lineal en la construcción de las PCA's